In [1]:
import torch
import torch.nn as nn
import sys
from tqdm import tqdm
from config import input_size, root, proposalN, channels
from utils.read_dataset import read_dataset
from utils.auto_laod_resume import auto_load_resume
from networks.model import MainNet
from torchvision import transforms

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
DEVICE = torch.device("cpu")


In [3]:
root = './datasets/FGVC-aircraft'  # dataset path
# model path
pth_path = "./models/air_epoch146.pth"
num_classes = 100

batch_size = 10

In [4]:
_, testloader = read_dataset(input_size, batch_size, root, "Aircraft")

Loading Aircraft trainset
Loading Aircraft testset


In [5]:
model = MainNet(proposalN=proposalN, num_classes=num_classes, channels=channels)

In [6]:
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

In [7]:
if os.path.exists(pth_path):
    epoch = auto_load_resume(model, pth_path, status='test')
else:
    sys.exit('There is not a pth exist.')

Load model from ./models/air_epoch146.pth
Resume from ./models/air_epoch146.pth


In [21]:
model.eval()

with torch.no_grad():
    data, target = next(iter(testloader))
    data, target = data.to(DEVICE), target.to(DEVICE)
    output = model(data, None, None, "test", "cpu")[-2:]
    pred = output[0].max(1, keepdim=True)[1]

    if pred.eq(target.view_as(pred)).sum().item() == 10:
        print("예측 성공")
    else:
        print("예측 실패")
    


        

예측 성공


In [19]:
raw_correct = 0
object_correct = 0
model.eval()

with torch.no_grad():
    for i, data in enumerate(tqdm(testloader)):
        x, y = data
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        local_logits, local_imgs = model(x, epoch, i, 'test', DEVICE)[-2:]
        # local
        pred = local_logits.max(1, keepdim=True)[1]
        object_correct += pred.eq(y.view_as(pred)).sum().item()

    print('\nObject branch accuracy: {}/{} ({:.2f}%)\n'.format(
        object_correct, len(testloader.dataset), 100. * object_correct / len(testloader.dataset)))

  4%|▍         | 15/334 [00:57<20:18,  3.82s/it]

KeyboardInterrupt: 